<a href="https://colab.research.google.com/github/mukerem/AmharicHandwrittenDigitRecognitionCNN/blob/main/Handwritten_Amharic_Digit_Recognition_Using_Deep_Learning-8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# **Load library**

In [ ]:
# !unzip '/content/gdrive/MyDrive/PreProcessV1.zip' -d "/content/gdrive/MyDrive/"

In [3]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
import seaborn as sns
import random
import warnings

from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras.backend as K
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau




config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
sess = tf.compat.v1.Session(config=config) 
K.set_session(sess)


warnings.filterwarnings('ignore')

In [4]:
%cd /content/gdrive/MyDrive/PreProcessV1/

/content/gdrive/MyDrive/PreProcessV1


In [5]:
# %cd /content/gdrive/MyDrive/PreProcessDataset/

In [6]:
batch_size = 32
img_height = 32
img_width = 32

In [7]:
data_dir = "/content/gdrive/MyDrive/PreProcessV1/"
# % pwd

In [8]:
# data_dir = "/content/gdrive/MyDrive/PreProcessDataset/"

### **keras**


In [9]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  color_mode="grayscale",
  batch_size=batch_size)

Found 51952 files belonging to 20 classes.
Using 41562 files for training.


In [10]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  color_mode="grayscale",
  batch_size=batch_size)

Found 51952 files belonging to 20 classes.
Using 10390 files for validation.


In [ ]:
# class_names = train_ds.class_names
# print(class_names)

In [12]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy())
#     plt.title(class_names[labels[i]])
#     plt.axis("off")

In [13]:
# for image_batch, labels_batch in train_ds:
#   print(image_batch.shape)
#   print(labels_batch.shape)
#   break

In [14]:
# normalization_layer = tf.keras.layers.Rescaling(1./255)

In [15]:
# normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
# image_batch, labels_batch = next(iter(normalized_ds))
# first_image = image_batch[0]
# # Notice the pixel values are now in `[0,1]`.
# print(np.min(first_image), np.max(first_image))

In [16]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [17]:
# model = keras.Sequential([
#             keras.Input(shape = (32, 32, 1)),
#             layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
#             layers.MaxPooling2D(),
#             layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
#             layers.MaxPooling2D(),
#             layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
#             layers.MaxPooling2D(),
#             layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
#             layers.MaxPooling2D(),
#             layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
#             layers.MaxPooling2D(),
#             layers.Conv2D(128, 3, padding = 'same', activation = 'relu'),
#             layers.Flatten(),
#             layers.Dense(128, activation = 'relu'),
#             layers.Dense(20, activation = 'softmax'),
# ])

In [18]:
# model = keras.Sequential([
#             keras.Input(shape = (32, 32, 1)),
#             layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
#             layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
#             layers.MaxPooling2D(),
#             layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
#             layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
#             layers.MaxPooling2D(),
#             layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
#             layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
#             layers.MaxPooling2D(),
#             layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
#             layers.MaxPooling2D(),
#             layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
#             layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
#             layers.MaxPooling2D(),
#             layers.Conv2D(128, 3, padding = 'same', activation = 'relu'),
#             layers.Flatten(),
#             layers.Dense(128, activation = 'relu'),
#             layers.Dense(20, activation = 'softmax'),
# ])

In [31]:
model = keras.Sequential([
            keras.Input(shape = (32, 32, 1)),
            layers.Conv2D(32, 3, padding = 'same', activation = 'relu', name="Conv1"),
            # layers.MaxPooling2D(name="Maxpool1"),
            layers.Conv2D(32, 3, padding = 'same', activation = 'relu', name="Conv2"),
            layers.MaxPooling2D(name="Maxpool1"),
            layers.Conv2D(32, 3, padding = 'same', activation = 'relu', name="Conv3"),
            layers.MaxPooling2D(name="Maxpool2"),
            layers.Conv2D(64, 3, padding = 'same', activation = 'relu', name="Conv4"),
            layers.MaxPooling2D(name="Maxpool3"),
            layers.Conv2D(64, 3, padding = 'same', activation = 'relu', name="Conv5"),
            # layers.MaxPooling2D(name="Maxpool1"),
            layers.Conv2D(64, 3, padding = 'same', activation = 'relu', name="Conv6"),
            layers.MaxPooling2D(name="Maxpool4"),
            layers.Conv2D(64, 3, padding = 'same', activation = 'relu', name="Conv7"),
            layers.MaxPooling2D(name="Maxpool5"),
            layers.Conv2D(128, 3, padding = 'same', activation = 'relu', name="Conv8"),
            layers.Flatten(name="Flatten1"),
            layers.Dense(128, activation = 'relu', name="Dense1"),
            layers.Dense(20, activation = 'softmax', name="Dense2"),
])

In [32]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv1 (Conv2D)              (None, 32, 32, 32)        320       
                                                                 
 Conv2 (Conv2D)              (None, 32, 32, 32)        9248      
                                                                 
 Maxpool1 (MaxPooling2D)     (None, 16, 16, 32)        0         
                                                                 
 Conv3 (Conv2D)              (None, 16, 16, 32)        9248      
                                                                 
 Maxpool2 (MaxPooling2D)     (None, 8, 8, 32)          0         
                                                                 
 Conv4 (Conv2D)              (None, 8, 8, 64)          18496     
                                                                 
 Maxpool3 (MaxPooling2D)     (None, 4, 4, 64)         

In [33]:
nadam = tf.keras.optimizers.Nadam(
    learning_rate=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam"
)

In [34]:
adamax = tf.keras.optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

In [35]:
sgd = tf.keras.optimizers.SGD(
    learning_rate=0.01,
    momentum=0.0,
    nesterov=False,
    name='SGD',
)

In [36]:
adam =  keras.optimizers.Adam(lr = 1e-3)

In [37]:
model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
        optimizer = adam,
        metrics = ['acc'],
)

In [38]:
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', patience = 2, verbose = 1, factor = 0.5, min_lr = 1e-7)

In [39]:
# classifier = model.fit_generator(datagen.flow(train_ds, batch_size = batch_size), 
#                                  epochs = 20,
#                                  callbacks=[reduce_lr],
#                                  validation_data = val_ds)

In [40]:
classifier = model.fit(train_ds, batch_size = batch_size, 
                                 epochs = 30,
                                 callbacks=[reduce_lr],
                                 validation_data = val_ds)

Epoch 1/30
1299/1299 [==============================] - 18s 13ms/step - loss: 1.3390 - acc: 0.5727 - val_loss: 0.6493 - val_acc: 0.8104 - lr: 0.0010
Epoch 2/30
1299/1299 [==============================] - 15s 12ms/step - loss: 0.4886 - acc: 0.8628 - val_loss: 0.4506 - val_acc: 0.8801 - lr: 0.0010
Epoch 3/30
1299/1299 [==============================] - 16s 12ms/step - loss: 0.3561 - acc: 0.9029 - val_loss: 0.4409 - val_acc: 0.8915 - lr: 0.0010
Epoch 4/30
1299/1299 [==============================] - 16s 13ms/step - loss: 0.3030 - acc: 0.9199 - val_loss: 0.5129 - val_acc: 0.8731 - lr: 0.0010
Epoch 5/30
1299/1299 [==============================] - 16s 12ms/step - loss: 0.2687 - acc: 0.9292 - val_loss: 0.3781 - val_acc: 0.9128 - lr: 0.0010
Epoch 6/30
1299/1299 [==============================] - 17s 13ms/step - loss: 0.2386 - acc: 0.9380 - val_loss: 0.4244 - val_acc: 0.9082 - lr: 0.0010
Epoch 7/30
1298/1299 [============================>.] - ETA: 0s - loss: 0.2203 - acc: 0.9427
Epoch 7: Redu

In [29]:
# model.save('/content/gdrive/MyDrive/30-epoch-val-acc-95-lr-1e-3-batch-32-adam.h5')

### **Data Augmentation**


In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
   rescale=1.0 / 255,
   rotation_range=30,
   width_shift_range=0.2,
   height_shift_range=0.2,
   vertical_flip=True,
)

In [ ]:

train_generator = train_datagen.flow_from_directory(
    directory=data_dir,
    target_size=(img_height, img_width),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 51952 images belonging to 20 classes.


In [ ]:
train_generator

In [ ]:
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
   rescale=1.0 / 255,
   rotation_range=30,
   width_shift_range=0.2,
   height_shift_range=0.2,
   vertical_flip=True,
)

In [ ]:
valid_generator = valid_datagen.flow_from_directory(
    directory=data_dir,
    target_size=(img_height, img_width),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 51952 images belonging to 20 classes.


In [ ]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
   rescale=1.0 / 255,
   rotation_range=30,
   width_shift_range=0.2,
   height_shift_range=0.2,
   vertical_flip=True,
)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    directory=data_dir,
    target_size=(img_height, img_width),
    color_mode="grayscale",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

Found 51952 images belonging to 20 classes.


In [ ]:
model = keras.Sequential([
            keras.Input(shape = (32, 32, 1)),
            layers.Conv2D(32, 3, padding = 'same', activation = 'relu', name="Conv1"),
            layers.Conv2D(32, 3, padding = 'same', activation = 'relu', name="Conv2"),
            layers.Conv2D(32, 3, padding = 'same', activation = 'relu', name="Conv3"),
            layers.MaxPooling2D(name="Maxpool1"),
            layers.Conv2D(64, 3, padding = 'same', activation = 'relu', name="Conv4"),
            layers.Conv2D(64, 3, padding = 'same', activation = 'relu', name="Conv5"),
            layers.Conv2D(64, 3, padding = 'same', activation = 'relu', name="Conv6"),
            layers.MaxPooling2D(name="Maxpool2"),
            layers.Conv2D(64, 3, padding = 'same', activation = 'relu', name="Conv7"),
            layers.Conv2D(128, 3, padding = 'same', activation = 'relu', name="Conv8"),
            layers.Flatten(name="Flatten1"),
            layers.Dense(128, activation = 'relu', name="Dense1"),
            layers.Dense(20, activation = 'softmax', name="Dense2"),
])

In [ ]:
adam =  keras.optimizers.Adam(lr = 1e-3)

In [ ]:
model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
        optimizer = adam,
        metrics = ['acc'],
)

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', patience = 2, verbose = 1, factor = 0.5, min_lr = 1e-7)

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

In [ ]:
classifier = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
                    callbacks=[reduce_lr]
)

InvalidArgumentError: ignored

In [ ]:
model.evaluate_generator(generator=valid_generator,
steps=STEP_SIZE_VALID)

# Pytorch Model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
transform = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Resize([32,32]),
    #  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

path = "/content/gdrive/MyDrive/PreProcessDataset"
batch_size = 128

trainset = torchvision.datasets.ImageFolder(path, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.conv5 = nn.Conv2d(64, 128, 3, padding=1)
        self.fc1 = nn.Linear(128 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 20)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool(F.relu(self.conv4(x)))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = torch.flatten(x, 1) 
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x))
        return x


net = Net()
net.to(device)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 10 == 9:    
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0

print('Finished Training')